# office.anyfive.com 이미지 태그 정리

## 작업 목표
- `documents` 테이블의 `doc_body`에서 `http://office.anyfive.com/` 도메인 이미지 태그 탐색
- 패턴: `<img[^>]+http://office\.anyfive\.com/[^>]+>`

## 작업 순서
1. 패턴 매칭 건수 확인 및 txt 파일 추출
2. txt 파일 검토 후 일괄 삭제 실행

---
## 1. DB 연결 설정

In [1]:
import pymysql
import re
from datetime import datetime

# DB 연결 정보 (수정 필요)
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval',
    'charset': 'utf8mb4'
}

# 패턴 정의
IMG_PATTERN = re.compile(r'<img[^>]+http://office\.anyfive\.com/[^>]+>', re.IGNORECASE)

print("설정 완료")

설정 완료


---
## 2. 패턴 탐색 및 txt 파일 추출

In [2]:
# DB에서 doc_body 조회 및 패턴 매칭
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# doc_body에 해당 패턴이 포함된 문서만 조회 (성능 위해 LIKE로 1차 필터링)
query = """
SELECT source_id, doc_body 
FROM documents 
WHERE doc_body LIKE '%http://office.anyfive.com/%'
"""

cursor.execute(query)
rows = cursor.fetchall()

print(f"1차 필터링 문서 수: {len(rows)}건")

# 패턴 매칭 및 결과 수집
results = []
for source_id, doc_body in rows:
    if doc_body:
        matches = IMG_PATTERN.findall(doc_body)
        for match in matches:
            results.append((source_id, match))

print(f"매칭된 이미지 태그 총 건수: {len(results)}건")
print(f"해당 문서 수: {len(set(r[0] for r in results))}건")

cursor.close()
conn.close()

1차 필터링 문서 수: 219건
매칭된 이미지 태그 총 건수: 219건
해당 문서 수: 210건


In [3]:
# txt 파일로 추출
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'office_anyfive_images_{timestamp}.txt'

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(f"# office.anyfive.com 이미지 태그 목록\n")
    f.write(f"# 추출일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"# 총 건수: {len(results)}건\n")
    f.write(f"# 문서 수: {len(set(r[0] for r in results))}건\n")
    f.write("#" + "="*80 + "\n\n")
    
    for source_id, img_tag in results:
        f.write(f"source_id: {source_id} | {img_tag}\n")

print(f"파일 저장 완료: {output_file}")
print(f"\n샘플 (처음 10건):")
for source_id, img_tag in results[:10]:
    print(f"  source_id: {source_id} | {img_tag[:80]}{'...' if len(img_tag) > 80 else ''}")

파일 저장 완료: office_anyfive_images_20251230_115350.txt

샘플 (처음 10건):
  source_id: 2009519 | <img border="0" height="305" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009557 | <img border="0" height="621" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009567 | <img border="0" height="298" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009568 | <img border="0" height="1008" src="http://office.anyfive.com/_data/CLOSE/anyfive...
  source_id: 2009591 | <img border="0" height="924" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009603 | <img border="0" height="238" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009610 | <img border="0" height="266" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009638 | <img border="0" height="481" src="http://office.anyfive.com/_data/CLOSE/anyfive/...
  source_id: 2009649 | <img border="0" height="302" src="http://office.anyfive

---
## 3. 일괄 삭제 (⚠️ 검토 후 실행)

**위에서 추출한 txt 파일을 먼저 검토한 후 실행하세요!**

In [4]:
# 삭제 대상 source_id 목록 (중복 제거)
target_source_ids = list(set(r[0] for r in results))
print(f"삭제 대상 문서 수: {len(target_source_ids)}건")
print(f"삭제 대상 이미지 태그 수: {len(results)}건")
print("\n⚠️ 아래 셀을 실행하면 실제로 삭제됩니다!")

삭제 대상 문서 수: 210건
삭제 대상 이미지 태그 수: 219건

⚠️ 아래 셀을 실행하면 실제로 삭제됩니다!


In [5]:
# ⚠️ 실제 삭제 실행 - 신중하게!

conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

updated_count = 0
error_count = 0

for source_id in target_source_ids:
    try:
        # 현재 doc_body 조회
        cursor.execute("SELECT doc_body FROM documents WHERE source_id = %s", (source_id,))
        row = cursor.fetchone()
        
        if row and row[0]:
            original_body = row[0]
            # 패턴 삭제
            cleaned_body = IMG_PATTERN.sub('', original_body)
            
            if original_body != cleaned_body:
                cursor.execute(
                    "UPDATE documents SET doc_body = %s WHERE source_id = %s",
                    (cleaned_body, source_id)
                )
                updated_count += 1
                
    except Exception as e:
        print(f"Error on source_id {source_id}: {e}")
        error_count += 1

conn.commit()
cursor.close()
conn.close()

print(f"\n완료!")
print(f"업데이트된 문서: {updated_count}건")
print(f"에러 발생: {error_count}건")


완료!
업데이트된 문서: 210건
에러 발생: 0건


---
## 4. 검증 (삭제 후 확인)

In [6]:
# 삭제 후 잔여 확인
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

cursor.execute("""
SELECT COUNT(*) 
FROM documents 
WHERE doc_body LIKE '%http://office.anyfive.com/%'
""")

remaining = cursor.fetchone()[0]
print(f"잔여 문서 수: {remaining}건")

cursor.close()
conn.close()

잔여 문서 수: 9건
